In [ ]:
!pip install repertorio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user
from google.auth import default
import gspread_dataframe as gd
import gspread
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
import os
os.chdir('/content/drive/MyDrive/Angela/SetlistScraping')
import pandas as pd
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In the below code:

1) Change **artists** to touring artists of interest, make sure links to correct artists are being found

2) update **date range** as desired

In [ ]:
import requests
from datetime import date
from bs4 import BeautifulSoup as bs4
import pandas as pd
from repertorio import Repertorio #API KEY: ad0Q8hXd28emWyBBPrHLpJWhx0trdUC08zZb
import time

##FUNCTIONS
def get_span_data(list, event, filter):
    """
    Input: uses BS4 to find tag data given the tag and the class_
    Output: appends retrieved location tag info to the list used as an argument
    """
    showcitytag = event.find('span', class_= filter)
    if showcitytag is not None:
        list.append(showcitytag.string)
    else:
        list.append(showcitytag)

agents = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
header = {
    'User-Agent': agents,
    'Accept-Language': 'en-US, en;q=0.5'
}

#only get songs for shows on or after this date
compdate = '2023-01-01' #TODO: CHANGE AS DESIRED
todaysdate = str(date.today())

#authenticate w/ API
api = Repertorio('ad0Q8hXd28emWyBBPrHLpJWhx0trdUC08zZb')

#TODO: CHANGE ONCE YOU HAVE LIST OF TOURING ARTISTS

# events = pd.read_csv('touringfile.csv', index_col=0) #initial process to get data from touring file spreadsheet vs. inputting artists manually
# artists = events['Artist Name']
# touringlist = list(artists)
inputlist = ['Sabrina Carpenter', 'Taylor Swift', 'blink-182', 'Sam Hunt', 'YUNGBLUD', 'Kendrick Lamar'] #,'kendrick lamar','kanye west'] #NOTE IF ARTIST NAME HAS CHARACTER LIKE '-' IN IT, OR IF ARTIST IS TOURING WITH ANOTHER ARTIST THESE RESULTS NEED TO BE CHECKED
testlist = [item.replace("-", "‐") for item in inputlist] #changes input from hyphen to hyphen-minus (aka en dash?), as Setlist API data is stored with en dashes (not intuitively input on keyboards)
links = []
for name in testlist: #for each touring artist, get url and add to links list
    link = ''
    correctName = False
    count = 1

    #get num results and check first page b4 entering while loop
    results = api.artists(artistName=name)
    df = pd.DataFrame(results['artist'])
    num_results = results['total']
    for index, row in df.iterrows():
        if row['name'].lower() == name.lower(): #TODO: NOTE SOME ARTISTS (ex. Drake) first matching result might not be correct person; if stuck in loop or something else, double check this first!
            link = row['url']
            correctName = True
        count += 1

    #if exact match not found on page 1
    page = 2
    while not correctName and count <= num_results:
        results = api.artists(artistName=name, p=page)
        df = pd.DataFrame(results['artist'])
        if len(df) == 0: #break out of loop if no results
            break
        for index, row in df.iterrows():
            if row['name'].lower() == name.lower():
                link = row['url']
                correctName = True
            count += 1
        page += 1
        time.sleep(5)

    links.append(link)
    time.sleep(5)

print(links)

#get data for each artist
allartistsdf = pd.DataFrame()

for artistlink in links:
    i = 1
    beforeDate = False
    showdates = []
    showcities = []
    showstates = []
    showcountries = []
    setlisturls = []

    #get artist
    artisturl = artistlink
    aristdata = requests.get(artisturl, headers=header)
    artistsoup = bs4(aristdata.text, 'html.parser')
    artist = artistsoup.find('h1', class_ = 'noMargin').contents[0].text[:-1] # takes extra space char off end

    #collect all show data between todays date and the comparison date
    while not beforeDate:
        showurl = f'{artisturl}?page={i}'
        showdata = requests.get(showurl, headers=header)
        showsoup = bs4(showdata.text, 'html.parser')
        events = showsoup.findAll('div', class_='col-xs-12 setlistPreview vevent')
        for event in events:
            #first get and compare show date
            showdatetag = event.find('span', class_ = 'value-title')
            if showdatetag is not None:
                showdate = showdatetag.get('title')
            else:
                showdate = None
            if showdate < compdate: #TODO: CHECK CASE WHEN THERE ARE NO SHOWS BEFORE COMPDATE?
                beforeDate = True
                break
            elif showdate >= todaysdate:
                continue # dont get data for future shows
            else:
                #then collect other data and append to lists
                showdates.append(showdate)

                get_span_data(showcities, event, 'locality')
                get_span_data(showstates, event, 'region')
                get_span_data(showcountries, event, 'country-name')

                setlisturltag = event.find('a', class_ = 'summary url')
                if setlisturltag is not None:
                    setlisturl = setlisturltag.get('href').replace('..', 'https://www.setlist.fm')
                    setlisturls.append(setlisturl)
                else:
                    setlisturls.append(setlisturltag)
        print(f'you have finished page {i}') #prints extra page at end after breaking, program ends on that page
        i += 1

    #create df with all tour/shows data for particular artist
    showsdf = pd.concat([pd.DataFrame(showdates),pd.DataFrame(showcities),pd.DataFrame(showstates),pd.DataFrame(showcountries),pd.DataFrame(setlisturls)],axis=1)
    showsdf.columns = ['Date', 'City', 'State', 'Country', 'setlistURL']
    showsdf = showsdf[showsdf['Country'] == 'USA'] #only get US shows
    showsdf = showsdf[showsdf['City'] != 'Unknown City'] #filter out cities that are unknown
    showsdf.dropna(inplace=True)
    showsdf.reset_index(inplace=True,drop=True)
    print(showsdf)

    finalsongs = pd.DataFrame()

    for i in range(len(showsdf)): #iterate through each show

        #empty lists to store show data
        numsongs = 0
        setlistsongs = []

        setlisturl = showsdf.iloc[i,4]
        setlistdata = requests.get(setlisturl, headers = header)
        setlistsoup = bs4(setlistdata.text, 'html.parser')
        songs = setlistsoup.findAll('li', class_ = 'setlistParts song')
        numsongs = len(songs)

        #get all songs for particular show
        for song in songs:
          setlistsongs.append(song.find('a', class_ = 'songLabel').string.replace("‐", "-").lower())
        setlistdf = pd.DataFrame(setlistsongs,columns=['Song'])

        #create df for one show, append to full df of all songs for all shows
        setlistdf['Date'] = showsdf.iloc[i,0]
        setlistdf['City'] = showsdf.iloc[i,1]
        setlistdf['State'] = showsdf.iloc[i,2]
        #setlistdf['Country'] = showsdf.iloc[i,3]
        finalsongs = pd.concat([finalsongs,setlistdf])

    finalsongs['Artist'] = artist.replace("‐", "-") #change all hyphen-minus' (from API formatting) to hyphens for artist/song names
    finalsongs['Song'] = finalsongs['Song'].replace("‐", "-")

    allartistsdf = pd.concat([allartistsdf,finalsongs])

print(allartistsdf)
#allartistsdf.to_csv('allsongs.csv') #TODO: CHANGE NAME TO DESIRED RESULT

['https://www.setlist.fm/setlists/sabrina-carpenter-4bdd3b6a.html', 'https://www.setlist.fm/setlists/taylor-swift-3bd6bc5c.html', 'https://www.setlist.fm/setlists/blink182-4bd68fae.html', 'https://www.setlist.fm/setlists/sam-hunt-6bd8268e.html', 'https://www.setlist.fm/setlists/yungblud-53cf3f09.html', 'https://www.setlist.fm/setlists/kendrick-lamar-bd2f5d6.html']
you have finished page 1
you have finished page 2
you have finished page 3
you have finished page 4
you have finished page 5
you have finished page 6
          Date            City State Country  \
0   2023-05-20     Gulf Shores    AL     USA   
1   2023-05-17         Atlanta    GA     USA   
2   2023-05-16         Raleigh    NC     USA   
3   2023-05-14      Washington    DC     USA   
4   2023-05-13    Philadelphia    PA     USA   
5   2023-05-11        New York    NY     USA   
6   2023-05-10        New York    NY     USA   
7   2023-05-09          Boston    MA     USA   
8   2023-05-07       Cleveland    OH     USA   
9  

In [ ]:
#read in csv and use it to create a gSheet
allsongs = allartistsdf #pd.read_csv('allsongs.csv', index_col=0)
book = 'allsongs' # this is the book/sheet currently linked to the Looker Dashboard table (fiveartistsetlists) the below query draws from
sheet = 'Sheet1'
ws = gc.open(book).worksheet(sheet)
ws.clear()

#overwrite allsongs Sheet currently linked to umg-us.stone_ume.allsetlists (console.cloud.google.com/bigquery?ws=!1m5!1m4!4m3!1sumg-us!2sstone_ume!3sallsetlists) to be new artists of interest
gd.set_with_dataframe(worksheet=ws,dataframe=allsongs,include_index=False,include_column_header=False,resize=False) #resize for changing dim to new dim

**BEFORE MOVING ON, MAKE SURE TO REFRESH [ALLSETLISTS](https://console.cloud.google.com/bigquery?ws=!1m5!1m4!4m3!1sumg-us!2sstone_ume!3sallsetlists) IN BIGQUERY.**

In the below code:

1) Update the **date range** of stream_subset subquery as desired (should match date range you specified in the second code block)

2) Make sure streaming_before and streaming_after draws FROM the correct, most recent table (should be already correct if you're just running these code blocks in order)

In [ ]:
# MAKE SQL CALL FROM PYTHON/COLAB

sql = f'''

WITH

  stream_subset AS ( -- this table is a subset of the full streaming data (60 TB -> a fraction of that)
    SELECT user_dma_name, isrc, stream_date
    FROM `umg-edw.spotify.streams`
    WHERE stream_date BETWEEN '2023-01-1' AND current_date() -- CHANGE THIS TO DESIRED START/END DATES!
    AND user_country_code = 'US'
  ),

  split_mashups AS ( -- this table accounts for mashup performances (aka artist performs song a + song b together and it's recorded on SetlistFM as one song: "song a / song b")
    SELECT t.*
    FROM umg-us.stone_ume.allsetlists t -- ("setlist") CHANGE DATASET BASED ON SETLISTS YOU'RE TRYING TO ANALYZE if necessary

    UNION ALL

    SELECT SPLIT(Song, ' / ')[OFFSET(0)] AS Song, Date, City, State, Artist
    FROM umg-us.stone_ume.allsetlists -- ("setlist") CHANGE DATASET BASED ON SETLISTS YOU'RE TRYING TO ANALYZE if necessary
    WHERE Song LIKE '% / %'

    UNION ALL

    SELECT SPLIT(Song, ' / ')[OFFSET(1)] AS Song, Date, City, State, Artist
    FROM umg-us.stone_ume.allsetlists -- ("setlist") CHANGE DATASET BASED ON SETLISTS YOU'RE TRYING TO ANALYZE if necessary
    WHERE Song LIKE '% / %'
  ),

  dmas AS (
    SELECT DISTINCT t.*, d.dma
    FROM split_mashups t
    JOIN (SELECT DISTINCT * FROM umg-us.stone_ume.city_to_DMA) d
    ON t.city = d.City AND t.state = d.State
  ),

  dates AS ( -- drops covers, non-artist songs, etc. by joining on artist AND song name
    SELECT a.*, m.track_isrc, CAST(a.Date AS DATE) AS day_of, DATE_ADD(CAST(a.Date AS DATE), INTERVAL 1 WEEK) AS week_later, DATE_SUB(CAST(a.Date AS DATE),INTERVAL 1 DAY) AS day_before, DATE_SUB(DATE_SUB(CAST(a.Date AS DATE),INTERVAL 1 DAY), INTERVAL 1 WEEK) AS week_before

    FROM dmas AS a
    JOIN umg-edw.spotify.track_metadata as m
    ON a.song = LOWER(m.track_name) AND LOWER(a.artist) = LOWER(m.artist_name)
  ),

  streaming_before AS (
    SELECT d.date, d.city, d.state, d.dma, d.song, d.artist, d.track_isrc, COUNT(s.isrc) as streams_before
    FROM stream_subset s
    JOIN (SELECT DISTINCT * FROM dates) d
    ON s.isrc = d.track_isrc
    AND s.stream_date BETWEEN d.week_before AND d.day_before
    AND UPPER(s.user_dma_name) LIKE d.dma || '%'
    GROUP BY d.date, d.city, d.state, d.dma, d.song, d.artist, d.track_isrc
  ),

  streaming_after AS (
    SELECT d.date, d.city, d.state, d.dma, d.song, d.artist, d.track_isrc, COUNT(s.isrc) as streams_after
    FROM stream_subset s
    JOIN (SELECT DISTINCT * FROM dates) d
    ON s.isrc = d.track_isrc
    AND s.stream_date BETWEEN d.day_of AND d.week_later
    AND UPPER(s.user_dma_name) LIKE d.dma || '%'
    GROUP BY d.date, d.city, d.state, d.dma, d.song, d.artist, d.track_isrc
  ),

  combined AS (
    SELECT b.date, b.city, b.state, b.dma, b.song, b.artist, b.track_isrc, streams_before, streams_after
    FROM streaming_before b
    JOIN streaming_after a
    ON b.date = a.date
    AND a.track_isrc = b.track_isrc
  ),

  plays AS (
    SELECT Artist, Song, COUNT(DISTINCT Date) AS plays
    FROM combined
    GROUP BY Artist, Song
  )

SELECT DISTINCT a.*, b.plays
FROM combined a
JOIN plays b
ON a.Artist = b.Artist
AND a.Song = b.Song

'''

project = 'umg-us'
sqlresults = pd.read_gbq(sql, project) #get results from BQ (use #header=True if one exists)
sqlresults

,date,city,state,dma,song,artist,track_isrc,streams_before,streams_after,plays
0,2023-07-08,Holmdel,NJ,NEW YORK,water under the bridge,Sam Hunt,USUM72207374,22782,12277,4
1,2023-07-01,Wallingford,CT,HARTFORD & NEW HAVEN,california,YUNGBLUD,USUG11800550,208,201,9
2,2023-07-06,Hartford,CT,HARTFORD & NEW HAVEN,body like a back road,Sam Hunt,USUM71700575,14289,9469,4
3,2023-07-08,Holmdel,NJ,NEW YORK,house party,Sam Hunt,USUM71411592,59557,28837,4
4,2023-07-08,Holmdel,NJ,NEW YORK,hard to forget,Sam Hunt,USUM72002308,22403,12063,4
...,...,...,...,...,...,...,...,...,...,...
4307,2023-05-26,East Rutherford,NJ,NEW YORK,delicate,Taylor Swift,USCJY1750007,199793,215409,41
4308,2023-05-28,East Rutherford,NJ,NEW YORK,style,Taylor Swift,USCJY1431319,335121,313754,41
4309,2023-05-27,East Rutherford,NJ,NEW YORK,love story,Taylor Swift,USCJY0803448,304,388,41
4310,2023-05-28,East Rutherford,NJ,NEW YORK,vigilante shit,Taylor Swift,USUG12205723,6353,11913,41


In [ ]:
#read in csv and use it to create a gSheet
tourstreams = sqlresults
book = 'SQLresults' # this is the book/sheet currently linked to the Looker Dashboard table
sheet = 'Sheet1'
ws = gc.open(book).worksheet(sheet)
ws.clear()

#overwrite SQLresults Sheet currently linked to umg-us.stone_ume.setliststreams (console.cloud.google.com/bigquery?ws=!1m5!1m4!4m3!1sumg-us!2sstone_ume!3ssetliststreams) to be new artists of interest
gd.set_with_dataframe(worksheet=ws,dataframe=tourstreams,include_index=False,include_column_header=False,resize=False) #resize for changing dim to new dim

Now, refresh [setliststreams](https://console.cloud.google.com/bigquery?ws=!1m5!1m4!4m3!1sumg-us!2sstone_ume!3ssetliststreams) and refresh the [Looker Dashboard](https://lookerstudio.google.com/reporting/345657b4-9209-43a8-bdd4-07bb7a2e083b) data connection. The Looker Dashboard should now be updated with data for the inputted artists/date range!

NOTE: check the map to make sure all cities are located within correct states/US. If not, adjust the formula in city_us accordingly to place dot markers in the correct locations.